In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


def load_data():
    mnist = datasets.fetch_openml("mnist_784", version=1, as_frame=False)
    X, y = mnist.data, mnist.target
    y = y.astype(np.int32)  # Convert target to integers
    return X, y


def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


def avg_pooling(image, pool_size):
    pooled_height = image.shape[0] // pool_size[0]
    pooled_width = image.shape[1] // pool_size[1]
    pooled_image = np.zeros((pooled_height, pooled_width))

    for i in range(pooled_height):
        for j in range(pooled_width):
            start_i, end_i = i * pool_size[0], (i + 1) * pool_size[0]
            start_j, end_j = j * pool_size[1], (j + 1) * pool_size[1]
            pooled_image[i, j] = np.mean(image[start_i:end_i, start_j:end_j])

    return pooled_image


def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

In [2]:
def train_model(X_train, y_train):
    model = LogisticRegression(
        max_iter=1000, solver="lbfgs", multi_class="multinomial", random_state=42
    )
    model.fit(X_train, y_train)
    return model

In [3]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


def predict(model, new_data):
    predictions = model.predict(new_data)
    return predictions

In [66]:
import tqdm

# Load and split the data
X, y = load_data()

pool_size = 7
X_pooled = np.zeros((X.shape[0], (28 // pool_size) * (28 // pool_size)))
for i in tqdm.tqdm(range(X.shape[0])):
    image = X[i].reshape(28, 28)
    pooled_image = avg_pooling(image, (pool_size, pool_size))
    X_pooled[i] = pooled_image.flatten()

X_train, X_test, y_train, y_test = split_data(X_pooled, y)

# Preprocess the data
X_train_scaled, X_test_scaled = preprocess_data(X_train, X_test)

100%|██████████| 70000/70000 [00:04<00:00, 16695.79it/s]


In [67]:
# Train the model
model = train_model(X_train_scaled, y_train)

/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [68]:
# Evaluate the model
evaluate_model(model, X_test_scaled, y_test)

Accuracy: 0.7457142857142857
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.68      0.74      1343
           1       0.82      0.90      0.86      1600
           2       0.82      0.75      0.79      1380
           3       0.79      0.77      0.78      1433
           4       0.64      0.69      0.66      1295
           5       0.68      0.61      0.64      1273
           6       0.84      0.87      0.86      1396
           7       0.77      0.81      0.79      1503
           8       0.64      0.69      0.66      1357
           9       0.64      0.64      0.64      1420

    accuracy                           0.75     14000
   macro avg       0.74      0.74      0.74     14000
weighted avg       0.75      0.75      0.75     14000

Confusion Matrix:
 [[ 916   10   23   23   19   77   23    4  237   11]
 [   0 1446    6   36   13   50    3   10   24   12]
 [  11   33 1038   66   60   11   81   12   54   14]
 [  22   5

In [69]:
# Example prediction
sample_data = X_test_scaled[:5]  # Predicting for the first 5 test samples
predictions = predict(model, sample_data)
print("Sample Predictions:", predictions)

Sample Predictions: [8 9 5 7 7]


In [70]:
import numpy as np


class LinUCB:
    def __init__(self, n_arms, n_features, alpha):
        self.alpha = alpha
        self.M = np.stack([np.eye(n_features) for _ in range(n_arms)])
        self.b = np.zeros((n_arms, n_features))  # Reward vector
        self.theta = np.random.uniform(
            -1, 1, (n_arms, n_features)
        )  # Parameter estimate
        self.arms = n_arms

    def predict(self, feature_vectors):
        results = []
        for i in range(len(feature_vectors)):
            results.append(self.select_arm(feature_vectors[i]))
        return results

    def select_arm(self, x):
        p_values = []
        for k in range(self.arms):
            p = x.dot(self.theta[k]) + self.alpha * np.sqrt(
                x.T.dot(np.linalg.inv(self.M[k])).dot(x)
            )
            p_values.append(p)
        return np.argmax(p_values)

    def update(self, chosen_arm, reward, x):
        self.M[chosen_arm] += np.outer(x, x)
        self.b[chosen_arm] += reward * x
        self.theta[chosen_arm] = np.linalg.inv(self.M[chosen_arm]).dot(
            self.b[chosen_arm]
        )

In [ ]:
import numpy as np


class BayesianBandit:
    def __init__(self, n_arms, n_features):
        self.M = np.stack([np.eye(n_features) for _ in range(n_arms)])
        self.b = np.zeros((n_arms, n_features))  # Reward vector
        self.theta = np.random.uniform(
            -1, 1, (n_arms, n_features)
        )  # Parameter estimate
        self.arms = n_arms
        self.gen = np.random.default_rng()

    def predict(self, feature_vectors):
        results = []
        for i in range(len(feature_vectors)):
            results.append(self.select_arm(feature_vectors[i]))
        return results

    def select_arm(self, x):
        p_values = []
        for k in range(self.arms):
            theta = self.gen.multivariate_normal(
                mean=self.theta[k],
                cov=np.linalg.inv(self.M[k]),
            )
            p_values.append(x.dot(theta))
        return np.argmax(p_values)

    def update(self, chosen_arm, reward, x):

        self.M[chosen_arm] += np.outer(x, x)
        self.b[chosen_arm] += reward * x
        self.theta[chosen_arm] = np.linalg.inv(self.M[chosen_arm]).dot(
            self.b[chosen_arm]
        )

In [101]:
# Example usage
n_features = X_train_scaled.shape[1]
alpha = 0.01
n_arms = 10  # Number of classes
# bandit = LinUCB(n_arms, n_features, alpha)
bandit = BayesianBandit(n_arms, n_features)

In [ ]:
for t in tqdm.tqdm(range(X_train_scaled.shape[0])):
    t = t % X_train_scaled.shape[0]
    arm = bandit.select_arm(X_train_scaled[t])
    bandit.update(arm, arm == y_train[t], X_train_scaled[t])

100%|██████████| 56000/56000 [00:33<00:00, 1659.17it/s]


In [103]:
evaluate_model(bandit, X_test, y_test)

Accuracy: 0.48442857142857143
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.66      0.66      1343
           1       0.00      0.00      0.00      1600
           2       0.82      0.51      0.63      1380
           3       0.51      0.69      0.59      1433
           4       0.46      0.62      0.53      1295
           5       0.88      0.18      0.29      1273
           6       0.77      0.64      0.70      1396
           7       0.94      0.05      0.10      1503
           8       0.31      0.84      0.45      1357
           9       0.36      0.74      0.48      1420

    accuracy                           0.48     14000
   macro avg       0.57      0.49      0.44     14000
weighted avg       0.56      0.48      0.43     14000

Confusion Matrix:
 [[ 890    0    3    7   19    0   22    0  360   42]
 [   3    0   91  348   61    8    1    0  798  290]
 [  43    0  705   75  147    1  149    0  212   48]
 [  63   

/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [ ]:
print(bandit.theta)

[[ 7.64416670e-03 -5.47566014e-02  7.94221620e-02 -6.32814698e-02
  -2.60378827e-02  5.87399915e-02 -3.34713155e-02  1.01829947e-01
   1.00217105e-01  2.82520469e-03 -8.37092661e-02  7.07094046e-02
  -3.78661370e-02  8.44014153e-02  4.00388069e-02 -2.02879070e-02]
 [-4.96920279e-03 -5.83447407e-02  5.72531527e-02  6.92781584e-03
  -3.32871012e-02 -1.97555380e-01  7.15063026e-02  2.81906254e-02
  -6.90593441e-02 -6.66960049e-02 -1.06153445e-01  4.76237535e-02
  -3.37024712e-02 -9.77438391e-02 -9.30104342e-04 -1.28853281e-02]
 [-3.22163185e-02  9.62712686e-02  2.57597709e-02 -2.39904127e-02
   8.79860568e-03 -2.23192198e-01 -9.09149121e-04 -1.78405079e-02
   4.89193113e-02  1.09485922e-01  6.85939896e-02  1.97018816e-02
   2.23936761e-02  5.90409451e-03  1.59842388e-02  4.58755956e-02]
 [ 2.01040236e-02  8.71105346e-02  2.44022703e-02 -2.60485476e-02
  -2.75120396e-02 -1.10923006e-01  4.79144154e-02 -6.29550908e-02
   2.22620786e-02 -1.64094450e-01  7.63795482e-02  2.08472553e-02
   6.38